# Bayer2RGB Usage Demonstration For Other Networks

With this notebook, you can use a Bayer2RGB model and its quantized checkpoint for debayerization of RGB image and see the accuracy difference between bilinear interpolation and Bayer2RGB debayered image on the "ai87net-imagenet-effnetv2" model.

In [1]:
###################################################################################################
#
# Copyright (C) 2023-2024 Analog Devices, Inc. All Rights Reserved.
#
# Analog Devices, Inc. Default Copyright Notice:
# https://www.analog.com/en/about-adi/legal-and-risk-oversight/intellectual-property/copyright-notice.html
#
###################################################################################################import cv2
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import torch
from torch.utils import data
import cv2


%matplotlib inline

sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'models'))

import ai8x
import parse_qat_yaml

from train import update_old_model_params


In [2]:
sys.path.append(os.path.join(os.getcwd(), './models/'))

In [3]:
directory = os.getcwd()
training_dir = os.path.abspath(os.path.join(directory, os.pardir))

In [4]:
b2rgb = importlib.import_module("ai85net-bayer2rgbnet")

In [5]:
class Args:
    def __init__(self, act_mode_8bit):
        self.act_mode_8bit = act_mode_8bit
        self.truncate_testset = False

In [6]:
act_mode_8bit = True # For evaluation mode, input/output range: -128, 127

test_batch_size = 1

args = Args(act_mode_8bit=act_mode_8bit)

checkpoint_path_b2rgb = os.path.abspath(os.path.join(training_dir, os.pardir, 'ai8x-synthesis', 'trained', 'ai85-bayer2rgb-qat8-q.pth.tar'))

qat_yaml_file_used_in_training_b2rgb = '../policies/qat_policy_imagenet.yaml'

ai_device = 87
round_avg = True

# imagenet

In [7]:
from datasets import imagenet
test_model = importlib.import_module('models.ai87net-imagenet-effnetv2')
data_path = '/data_ssd/'
checkpoint_path = os.path.abspath(os.path.join(training_dir, os.pardir, 'ai8x-synthesis', 'trained', 'ai87-imagenet-effnet2-q.pth.tar'))
qat_yaml_file_used_in_training =  os.path.abspath(os.path.join(training_dir, 'policies', 'qat_policy_imagenet.yaml'))

# Dataset used for Biliner Interpolation
_, test_set_inter = imagenet.imagenet_bayer_fold_2_get_dataset((data_path, args), load_train=False, load_test=True, fold_ratio=1)

# Dataset used for Bayer2RGB Debayerization
_, test_set = imagenet.imagenet_bayer_fold_2_get_dataset((data_path, args), load_train=False, load_test=True, fold_ratio=2)

# Original dataset
_, test_set_original = imagenet.imagenet_get_datasets((data_path, args), load_train=False, load_test=True)


In [8]:
test_dataloader_inter = data.DataLoader(test_set_inter, batch_size=test_batch_size, shuffle=False)
test_dataloader = data.DataLoader(test_set, batch_size=test_batch_size, shuffle=False)
test_dataloader_original = data.DataLoader(test_set_original, batch_size=test_batch_size, shuffle=False)
print(len(test_dataloader))
print(len(test_dataloader.dataset))

50000
50000


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

qat_policy_bayer2rgb = parse_qat_yaml.parse(qat_yaml_file_used_in_training_b2rgb)
qat_policy = parse_qat_yaml.parse(qat_yaml_file_used_in_training)

ai8x.set_device(device=ai_device, simulate=act_mode_8bit, round_avg=round_avg)

model_bayer2rgb = b2rgb.bayer2rgbnet().to(device)


Configuring device: MAX78002, simulate=True.


Run one of the following models according to the dataset.

In [10]:
model = test_model.AI87ImageNetEfficientNetV2(bias="--use-bias").to(device)

In [11]:
model.state_dict()

OrderedDict([('conv_stem.output_shift', tensor([0.], device='cuda:0')),
             ('conv_stem.activation_threshold', tensor(0., device='cuda:0')),
             ('conv_stem.final_scale', tensor(0., device='cuda:0')),
             ('conv_stem.weight_bits', tensor([0], device='cuda:0')),
             ('conv_stem.bias_bits', tensor([0], device='cuda:0')),
             ('conv_stem.quantize_activation',
              tensor([False], device='cuda:0')),
             ('conv_stem.clamp_activation', tensor([False], device='cuda:0')),
             ('conv_stem.adjust_output_shift',
              tensor([False], device='cuda:0')),
             ('conv_stem.shift_quantile', tensor([1.], device='cuda:0')),
             ('conv_stem.op.weight',
              tensor([[[[ 1.0647e-01, -4.1215e-02,  6.5497e-02],
                        [ 1.2874e-01,  5.6039e-02,  1.2118e-01],
                        [ 1.1892e-01, -8.9278e-03,  2.7363e-01]],
              
                       [[ 2.5582e-02, -3.6456e-02,

In [12]:
model_bayer2rgb.state_dict()

OrderedDict([('l1.output_shift', tensor([0.], device='cuda:0')),
             ('l1.activation_threshold', tensor(0., device='cuda:0')),
             ('l1.final_scale', tensor(0., device='cuda:0')),
             ('l1.weight_bits', tensor([0], device='cuda:0')),
             ('l1.bias_bits', tensor([0], device='cuda:0')),
             ('l1.quantize_activation', tensor([False], device='cuda:0')),
             ('l1.clamp_activation', tensor([False], device='cuda:0')),
             ('l1.adjust_output_shift', tensor([False], device='cuda:0')),
             ('l1.shift_quantile', tensor([1.], device='cuda:0')),
             ('l1.op.weight',
              tensor([[[[-0.4293]],
              
                       [[ 0.3906]],
              
                       [[ 0.4348]],
              
                       [[ 0.2686]]],
              
              
                      [[[ 0.3106]],
              
                       [[-0.2797]],
              
                       [[-0.4760]],
 

In [13]:
# Trained checkpoints in synthesis repo was trained with earlier qat version. For compatibility, update checkpoint:
update_old_model_params(checkpoint_path_b2rgb, model_bayer2rgb)
update_old_model_params(checkpoint_path, model)


# fuse the BN parameters into conv layers before Quantization Aware Training (QAT)
ai8x.fuse_bn_layers(model_bayer2rgb)
ai8x.fuse_bn_layers(model)

checkpoint_b2rgb = torch.load(checkpoint_path_b2rgb,map_location = device)
checkpoint = torch.load(checkpoint_path,map_location = device)

# switch model from unquantized to quantized for QAT
ai8x.initiate_qat(model_bayer2rgb, qat_policy_bayer2rgb)
ai8x.initiate_qat(model, qat_policy)

model_bayer2rgb.load_state_dict(checkpoint_b2rgb['state_dict'], strict=False)
model.load_state_dict(checkpoint['state_dict'], strict=False)

ai8x.update_model(model_bayer2rgb)
model_bayer2rgb = model_bayer2rgb.to(device)
ai8x.update_model(model)
model = model.to(device)

Model /home/seldauyanik/fresh_clones/ai8x-synthesis/trained/ai85-bayer2rgb-qat8-q.pth.tar is old. Missing parameters added with default values!


# Bayer-to-RGB + AI87ImageNetEfficientNetV2 Model
Bayer2RGB model is used before AI87ImageNetEfficientNetV2 to obtain RGB images from bayered images and then AI87ImageNetEfficientNetV2 model is evaluated.

In [14]:
model_bayer2rgb.eval()
model.eval()
correct = 0
with torch.no_grad():
    for (image1, label1), (image2, label2) in zip(test_dataloader, test_dataloader_original):
        image = image1.to(device)

        primer_out = model_bayer2rgb(image)

        model_out = model(primer_out)
        result = np.argmax(model_out.cpu())

        if(label2 == result):
            correct = correct + 1 
        if correct % 15 == 0:
            print("accuracy:")
            print(correct / len(test_set))

print("accuracy:")
print(correct / len(test_set))

accuracy:
0.0
accuracy:
0.0003
accuracy:
0.0006
accuracy:
0.0009
accuracy:
0.0012
accuracy:
0.0012
accuracy:
0.0015
accuracy:
0.0015
accuracy:
0.0018
accuracy:
0.0021
accuracy:
0.0021
accuracy:
0.0024
accuracy:
0.0027
accuracy:
0.0027
accuracy:
0.0027
accuracy:
0.0027
accuracy:
0.003
accuracy:
0.0033
accuracy:
0.0033
accuracy:
0.0036
accuracy:
0.0039
accuracy:
0.0042
accuracy:
0.0042
accuracy:
0.0045
accuracy:
0.0045
accuracy:
0.0045
accuracy:
0.0048
accuracy:
0.0048
accuracy:
0.0048
accuracy:
0.0048
accuracy:
0.0051
accuracy:
0.0054
accuracy:
0.0057
accuracy:
0.006
accuracy:
0.0063
accuracy:
0.0063
accuracy:
0.0066
accuracy:
0.0066
accuracy:
0.0066
accuracy:
0.0069
accuracy:
0.0072
accuracy:
0.0075
accuracy:
0.0078
accuracy:
0.0081
accuracy:
0.0084
accuracy:
0.0084
accuracy:
0.0087
accuracy:
0.009
accuracy:
0.0093
accuracy:
0.0096
accuracy:
0.0099
accuracy:
0.0102
accuracy:
0.0105
accuracy:
0.0108
accuracy:
0.0111
accuracy:
0.0111
accuracy:
0.0111
accuracy:
0.0114
accuracy:
0.0117
acc

KeyboardInterrupt: 

# Model
Original Dataset is used to evaluate AI87ImageNetEfficientNetV2 model.

In [ ]:
model.eval()
correct = 0
with torch.no_grad():
    for image, label in test_dataloader_original:
        image = image.to(device)
        model_out = model(image)
        result = np.argmax(model_out.cpu())

        if(label == result):
            correct = correct + 1

        if correct % 15 == 0:
            print("accuracy:")
            print(correct / len(test_set))

print("accuracy:")
print(correct / len(test_set))

# Bilinear Interpolation + Model
Bilinear Interpolation is used before AI87ImageNetEfficientNetV2 to obtain RGB images from bayered images and then  model is evaluated.

In [ ]:
model.eval()
correct = 0
with torch.no_grad():
    for (image1, label1), (image2, label2) in zip(test_dataloader_inter, test_dataloader_original):
        image = image1.to(device)

        img = (128+(image[0].cpu().detach().numpy().transpose(1,2,0))).astype(np.uint8)
        img = cv2.cvtColor(img,cv2.COLOR_BayerGR2RGB)

        out_tensor = torch.Tensor(((img.transpose(2,0,1).astype(np.float32))/128-1)).to(device)
        out_tensor = out_tensor.unsqueeze(0)
        model_out = model(out_tensor)
        result = np.argmax(model_out.cpu())

        if(label2 == result):
            correct = correct + 1

        if correct % 15 == 0:
            print("accuracy:")
            print(correct / len(test_set))

print("accuracy:")
print(correct / len(test_set))